This is a follow up of the preprocessing notebook, written by Arad HamidSamiee.

---

#Earthquake Magnitude Prediction#
(part 3 / 3)

In [389]:
!rm -rf BaseData

In [379]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, regularizers
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator # will be utelized to label images

In [ ]:
!unzip BaseData.zip

In [ ]:
train = ImageDataGenerator()
validation = ImageDataGenerator()

train_dataset = train.flow_from_directory(directory = "BaseData/train/",
                                          target_size = (256,256),
                                          batch_size = 3,
                                          class_mode = 'binary')

validation_dataset = train.flow_from_directory(directory = "BaseData/validation/",
                                          target_size = (256,256),
                                          batch_size = 3,
                                          class_mode = 'binary')

In [ ]:
print(train_dataset.class_indices, validation_dataset.class_indices)

In [ ]:
train_dataset.classes

In [384]:
model = models.Sequential([
                           layers.Conv2D(filters=8, kernel_size=(5,5), strides=(1,1), activation='relu'),
                           layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
                           
                           layers.Conv2D(filters=8, kernel_size=(5,5), strides=(1,1), activation='relu'),
                           layers.AvgPool2D(pool_size=(3,3), strides=(2,2)),

                           layers.Conv2D(filters=8, kernel_size=(5,5), strides=(1,1), activation='relu'),
                           layers.AvgPool2D(pool_size=(3,3), strides=(2,2)),

                           layers.Dropout(rate=0.5),
                           layers.Dropout(rate=0.5),

                           layers.Flatten(),

                           layers.Dense(512, 'relu', kernel_regularizer = regularizers.L2(0.0005), bias_regularizer = regularizers.L2(0.0005)),
                           layers.Dense(1,'relu', kernel_regularizer = regularizers.L2(0.0005), bias_regularizer = regularizers.L2(0.0005))
                           ])

In [385]:
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)
# L1 : laplace
# l2 : Gaussian
# L2 = Weight Decay

In [ ]:
model_fit = model.fit(train_dataset,
                      steps_per_epoch = 3,
                      epochs = 500,
                      validation_data = validation_dataset
                      )

In [ ]:
validation_dataset.class_indices

In [ ]:
TestDataPath = 'BaseData/test'
for i in os.listdir(TestDataPath):
    img = image.load_img(TestDataPath+'//'+i, target_size=(256,256))
    plt.imshow(img)
    plt.show()
    print(TestDataPath+'//'+i)
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])
    label = model.predict(images)
    if label == 1:
        print(math.floor(label[0][0]), "Oh oh, That's BAD news! O_o \n")
    else:
        print(math.floor(label[0][0]), "You'll be fine.\n")
    # elif label == 1:
    #     print(label, "Oh oh, That's BAD news!\n")
    # else:
    #     print(label,"\nABOMINATION!!!\n")